In [ ]:
import numpy as np
import xarray as xr
import geopandas as gpd
import pandas as pd
from osgeo import gdal, ogr
import os
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
directory = "/g/data/r78/cb3058/dea-notebooks/ICE_project/results/nmdb_test/"

In [ ]:
#lits of years to help for loop iterate through folders
x = range(1987,2019,1)
years = []
for i in x:
    nextyear = str(i + 1)[2:]
    y = str(i) + "_" + nextyear
    years.append(str(y))
# removing years that didn't work
years =  [e for e in years if e not in ('1988_89', '1991_92', '1993_94', '2015_16', '2011_12', '2012_13')]
years.sort()

folders = os.listdir(directory)
folders.sort()

area_a = []
area_b = []

#grab sum of irragted area for each each and add to dataframe
for year, folder in zip(years, folders): 
    #progress indicator
    print("\r", "working on year: " + year, end = '')
    def getIrrigatedArea(shp):
        irr = gpd.read_file(shp)
        a = irr[irr.DN==80]
        b = irr[(irr.DN==80) | (irr.DN==75)]
        area_a.append(a.area.sum() / 10000)
        area_b.append(b.area.sum() / 10000)

    getIrrigatedArea(directory+folder+"/"+"nmdb_Summer"+ year + "_Irrigated.shp")

#convert years back into integers for plot
years_dt = []
for i in years:
    x = int(i[:-3])
    years_dt.append(x)
#create pandas dataframe
df = pd.DataFrame.from_dict({'year':years_dt, '0.8':area_a, '0.75+0.8':area_b})
df = df.set_index('year')

#plot
ax = df.plot(colormap='jet', linestyle='--', marker='o', figsize=(10,10))
ax.set_xlabel('Year (Summer Nov-Mar)')
ax.set_ylabel('Area under Irrigation (Ha)')

### Create an 'irrigatable area' mask by unioning all '0.8' polygons
    

In [ ]:
# shp1= "/g/data/r78/cb3058/dea-notebooks/ICE_project/results/nmdb_test/nmdb_Summer1987_88/nmdb_Summer1987_88_Irrigated.shp"
# shp2= "/g/data/r78/cb3058/dea-notebooks/ICE_project/results/nmdb_test/nmdb_Summer1989_90/nmdb_Summer1989_90_Irrigated.shp"

# irr1 = gpd.read_file(shp1)
# irr2 = gpd.read_file(shp2)
# irr3 = gpd.read_file(shp3)

# a = irr1[irr1.DN==80]
# b = irr2[irr2.DN==80]

# test = gpd.overlay(a,b, how='union')
# len(test)
# # test.plot(figsize=(20,20))

In [ ]:
x = range(1987,2019,1)
years = []
for i in x:
    nextyear = str(i + 1)[2:]
    y = str(i) + "_" + nextyear
    years.append(str(y))

years =  [e for e in years if e not in ('1988_89', '1991_92', '1993_94', '2015_16', '2011_12', '2012_13')]
years.sort()
    
directory = "/g/data/r78/cb3058/dea-notebooks/ICE_project/results/nmdb_test/"    
folders = os.listdir(directory)
folders.sort()
# shapes= []

#go through shapes, index for 0.8 threshold and export to shapefiles
for year, folder in zip(years, folders): 
    def select80polys(shp):
        print("\r", "working on year: " + year, end = '')
        irr = gpd.read_file(shp)
        a = irr[irr.DN==80]
        a.to_file('results/nmdb_union/80/'+year+".shp")
    
    select80polys(directory+folder+"/"+"nmdb_Summer"+ year + "_Irrigated.shp")    

In [ ]:
#union all of the filtered shapefiles
folder = Path("results/nmdb_union/80/")
shapes = folder.glob("*.shp")
gdf = pd.concat([
    gpd.read_file(shp)
    for shp in shapes
]).pipe(gpd.GeoDataFrame)
gdf.to_file("results/nmdb_union/union.shp")

In [ ]:
shp = "results/nmdb_union/union.shp"
union = gpd.read_file(shp)

In [ ]:
diss = union.dissolve(by='DN', aggfunc='sum')

In [ ]:
diss.head()

In [ ]:
diss.to_file('results/ndmb_union/dissolve.shp')
